In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import wb
import pycountry

import altair as alt

### WORLD BANK GDP PER CAPITA (PPP)

#### COUNTRIES

In [2]:
# Former USSR countries
fsu_countries = [
    {'name': 'Russia', 'region': 'Eastern Europe'},
    {'name': 'Ukraine', 'region': 'Eastern Europe'},
    {'name': 'Belarus', 'region': 'Eastern Europe'},
    {'name': 'Lithuania', 'region': 'Baltic states'},
    {'name': 'Latvia', 'region': 'Baltic states'},
    {'name': 'Estonia', 'region': 'Baltic states'},
    {'name': 'Moldova', 'region': 'Eastern Europe'},
    
    {'name': 'Uzbekistan', 'region': 'Central Asia'},
    {'name': 'Kazakhstan', 'region': 'Central Asia'},
    {'name': 'Kyrgyzstan', 'region': 'Central Asia'},
    {'name': 'Tajikistan', 'region': 'Central Asia'},
    {'name': 'Turkmenistan', 'region': 'Central Asia'},
    
    {'name': 'Georgia', 'region': 'South Caucasus'},
    {'name': 'Azerbaijan', 'region': 'South Caucasus'},
    {'name': 'Armenia', 'region': 'South Caucasus'},
]

In [3]:
# Former Warsaw Treaty/Pact countries, excluding East Germany
fwt_countries = [
    {'name': 'Bulgaria', 'region': 'Eastern Europe'},
    {'name': 'Slovakia', 'region': 'Eastern Europe'},
    {'name': 'Czech Republic', 'region': 'Eastern Europe'},
    {'name': 'Hungary', 'region': 'Eastern Europe'},
    {'name': 'Poland', 'region': 'Eastern Europe'},
    {'name': 'Romania', 'region': 'Eastern Europe'},
]

In [4]:
# Add ISO 3 codes
for rec in fsu_countries:
    rec['iso'] = pycountry.countries.search_fuzzy(
        rec['name']
    )[0].alpha_3
    
    rec['group'] = 'FSU'
    
for rec in fwt_countries:
    rec['iso'] = pycountry.countries.search_fuzzy(
        rec['name']
    )[0].alpha_3
    
    rec['group'] = 'FWT'

#### DOWNLOAD DATA

In [5]:
iso_codes = list(map(lambda x: x['iso'], fsu_countries + fwt_countries))
use_cols = [
    'Country Name',
    'Country Code',
    'Indicator Code'
] + list(map(lambda x: str(x), range(1990, 2021)))

In [6]:
# GDP per capita growth (annual %)
gdp_pc_grw = pd.read_csv(
    'data/API_NY.GDP.PCAP.KD.ZG_DS2_en_csv_v2_3634329.csv',
    sep=',',
    skiprows=4,
    usecols = use_cols
)

gdp_pc_grw = gdp_pc_grw.loc[
    gdp_pc_grw['Country Code'].isin(iso_codes + ['EUU', 'ECS', 'ECA'])
]

In [7]:
# GDP per capita, PPP (constant 2017 international $)
gdp_pc_ppp = pd.read_csv(
    'data/API_NY.GDP.PCAP.PP.KD_DS2_en_csv_v2_3682444.csv',
    sep=',',
    skiprows=4,
    usecols = use_cols
)

gdp_pc_ppp = gdp_pc_ppp.loc[
    gdp_pc_ppp['Country Code'].isin(iso_codes + ['EUU', 'ECS', 'ECA'])
]

In [8]:
# Population, total
pop = pd.read_csv(
    'data/API_SP.POP.TOTL_DS2_en_csv_v2_3628828.csv',
    sep=',',
    skiprows=4,
    usecols = use_cols
)

pop = pop.loc[
    pop['Country Code'].isin(iso_codes + ['EUU', 'ECS', 'ECA'])
]

In [9]:
pop.head()

,Country Name,Country Code,Indicator Code,1990,1991,1992,1993,1994,1995,1996,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
10,Armenia,ARM,SP.POP.TOTL,3538164.0,3505249.0,3442820.0,3363111.0,3283664.0,3217349.0,3168213.0,...,2876536.0,2884239.0,2897593.0,2912403.0,2925559.0,2936147.0,2944789.0,2951741.0,2957728.0,2963234.0
15,Azerbaijan,AZE,SP.POP.TOTL,7175200.0,7271300.0,7382050.0,7494800.0,7596550.0,7684850.0,7763000.0,...,9173082.0,9295784.0,9416801.0,9535079.0,9649341.0,9757812.0,9854033.0,9939771.0,10024283.0,10093121.0
21,Bulgaria,BGR,SP.POP.TOTL,8718289.0,8632367.0,8540164.0,8472313.0,8443591.0,8406067.0,8362826.0,...,7348328.0,7305888.0,7265115.0,7223938.0,7177991.0,7127822.0,7075947.0,7025037.0,6975761.0,6934015.0
25,Belarus,BLR,SP.POP.TOTL,10189348.0,10194050.0,10216470.0,10239050.0,10226955.0,10193831.0,10159569.0,...,9461643.0,9446836.0,9443211.0,9448515.0,9461076.0,9469379.0,9458989.0,9438785.0,9419758.0,9379952.0
54,Czech Republic,CZE,SP.POP.TOTL,10333355.0,10308578.0,10319123.0,10329855.0,10333587.0,10327253.0,10315241.0,...,10496088.0,10510785.0,10514272.0,10525347.0,10546059.0,10566332.0,10594438.0,10629928.0,10671870.0,10698896.0


In [10]:
countries_df = pd.DataFrame(
    fwt_countries + fsu_countries
).rename(columns={
    'name': 'Country Name',
    'region': 'Region',
    'iso': 'Country Code',
    'group': 'Group'
})

In [11]:
# Merge with countries
pop = pop.merge(countries_df.loc[:, ['Group', 'Country Code', 'Region']], on = 'Country Code', how ='left')
gdp_pc_ppp = gdp_pc_ppp.merge(countries_df.loc[:, ['Group', 'Country Code', 'Region']], on = 'Country Code', how ='left')
gdp_pc_grw = gdp_pc_grw.merge(countries_df.loc[:, ['Group', 'Country Code', 'Region']], on = 'Country Code', how ='left')

In [12]:
# Wide to long format
pop = pd.melt(
    pop,
    id_vars = [
        'Country Name',
        'Country Code',
        'Indicator Code',
        'Group',
        'Region'
    ],
    var_name = 'Year',
    value_name = 'Value'
)

gdp_pc_ppp = pd.melt(
    gdp_pc_ppp,
    id_vars = [
        'Country Name',
        'Country Code',
        'Indicator Code',
        'Group',
        'Region'
    ],
    var_name = 'Year',
    value_name = 'Value'
)

gdp_pc_grw = pd.melt(
    gdp_pc_grw,
    id_vars = [
        'Country Name',
        'Country Code',
        'Indicator Code',
        'Group',
        'Region'
    ],
    var_name = 'Year',
    value_name = 'Value'
)

In [13]:
pop.loc[:, 'Year'] = pd.to_numeric(pop.Year)
gdp_pc_ppp.loc[:, 'Year'] = pd.to_numeric(gdp_pc_ppp.Year)
gdp_pc_grw.loc[:, 'Year'] = pd.to_numeric(gdp_pc_grw.Year)

### NAs

In [14]:
gdp_pc_ppp.loc[
    gdp_pc_ppp.Value.isna()
]

,Country Name,Country Code,Indicator Code,Group,Region,Year,Value
7,Estonia,EST,NY.GDP.PCAP.PP.KD,FSU,Baltic states,1990,NaN
10,Hungary,HUN,NY.GDP.PCAP.PP.KD,FWT,Eastern Europe,1990,NaN
13,Lithuania,LTU,NY.GDP.PCAP.PP.KD,FSU,Baltic states,1990,NaN
14,Latvia,LVA,NY.GDP.PCAP.PP.KD,FSU,Baltic states,1990,NaN
15,Moldova,MDA,NY.GDP.PCAP.PP.KD,FSU,Eastern Europe,1990,NaN
19,Slovak Republic,SVK,NY.GDP.PCAP.PP.KD,FWT,Eastern Europe,1990,NaN
31,Estonia,EST,NY.GDP.PCAP.PP.KD,FSU,Baltic states,1991,NaN
37,Lithuania,LTU,NY.GDP.PCAP.PP.KD,FSU,Baltic states,1991,NaN
38,Latvia,LVA,NY.GDP.PCAP.PP.KD,FSU,Baltic states,1991,NaN
39,Moldova,MDA,NY.GDP.PCAP.PP.KD,FSU,Eastern Europe,1991,NaN


In [15]:
gdp_pc_ppp.loc[
    gdp_pc_ppp.Value.isna(),
    'Country Name'
].value_counts()

Estonia            5
Moldova            5
Lithuania          5
Latvia             5
Slovak Republic    2
Turkmenistan       1
Hungary            1
Name: Country Name, dtype: int64

#### IMPUTE NAs WITH GAPMINDER DATA

In [16]:
gdp_pc_ppp_gm = pd.read_csv(
    'data/GM-GDP per capita - Dataset - v27 - data-GDP-per-capita-in-columns.csv',
    sep=',',
    skiprows=3,
    usecols = [
        'geo', 'Country Name', '1990', '1991', '1992', '1993', '1994', '2020'
    ]
)

In [17]:
fill_df = gdp_pc_ppp_gm.loc[
    gdp_pc_ppp_gm['Country Name'].isin(
        gdp_pc_ppp.loc[
            gdp_pc_ppp.Value.isna(),
            'Country Name'
        ]
    )
].melt(id_vars = ['Country Name', 'geo'], var_name = 'Year', value_name = 'Value')

fill_df.loc[:, 'Year'] = pd.to_numeric(fill_df.loc[:, 'Year'])

In [18]:
fill_df.dtypes

Country Name     object
geo              object
Year              int64
Value           float64
dtype: object

In [19]:
gdp_pc_ppp = gdp_pc_ppp.merge(
    fill_df.loc[:, ['Country Name', 'Year', 'Value']],
    on = ['Country Name', 'Year'],
    how = 'left',
    suffixes = ['', '_fill']
)

In [20]:
gdp_pc_ppp.loc[
    gdp_pc_ppp.Value.isna(),
    'Value'
] = gdp_pc_ppp.loc[
    gdp_pc_ppp.Value.isna(),
    'Value_fill'
]

In [21]:
gdp_pc_ppp.drop('Value_fill', axis=1, inplace=True)

In [22]:
gdp_pc_ppp.loc[
    gdp_pc_ppp.Value.isna()
]

,Country Name,Country Code,Indicator Code,Group,Region,Year,Value


### EDA

#### GDP PPC

In [23]:
df = gdp_pc_ppp.loc[
    gdp_pc_ppp.Year.isin([1991, 2020])
].copy()

# Add Pct Change and Labels
pct_change = df.groupby('Country Code').apply(
    lambda tx: tx.sort_values(by='Year').loc[:, ['Value']].pct_change().iloc[1] * 100
).rename(columns = {'Value': 'PctChange'})

df = df.merge(pct_change, on = 'Country Code')

df.loc[:, 'Label'] = df.loc[:, 'Country Name'] + ' (' + df.loc[:, 'PctChange'].round(2).astype(str) + '%)'
df.loc[df.PctChange>0, 'Label'] = df.loc[:, 'Country Name'] + ' (+' + df.loc[:, 'PctChange'].round(2).astype(str) + '%)'


x = 'Year:O'
y = 'Value'
detail = 'Country Code'

color_fwt = alt.Color(
    'Group',
    scale = alt.Scale(
    domain = ['FWT', 'FSU', 'null'],
    range = ['#D50000', '#E0E0E0', '#212121']
    ),
    legend=alt.Legend(orient="top")
)

color_fsu = alt.Color(
    'Group',
    scale = alt.Scale(
    domain = ['FWT', 'FSU', 'null'],
    range = ['#E0E0E0', '#D50000', '#212121']
    ),
    legend=alt.Legend(orient="top")
)

text = alt.condition(alt.datum.Year == 2020, 'Label', alt.value(' '))

width = 250
height = 550

# FWT

base_fwt = alt.Chart(
    df.loc[
        ~df['Country Code'].isin(['FSU', 'ECS', 'ECA'])
    ]
).encode(
    x=x,
    y=y,
    color=color_fwt,
    detail=detail,
    text=text
)

slope_fwt = alt.layer(
    base_fwt.mark_line(),
    base_fwt.mark_circle(),
    base_fwt.mark_text(
        align='left',
        baseline='middle',
        dx=15  # Nudges text to right so it doesn't appear on top of the bar
    )
).properties(
    width = width,
    height = height
)

# FSU

base_fsu = alt.Chart(
    df.loc[
        ~df['Country Code'].isin(['FWT', 'ECS', 'ECA'])
    ]
).encode(
    x=x,
    y=y,
    color=color_fsu,
    detail=detail,
    text=text
)

slope_fsu = alt.layer(
    base_fsu.mark_line(),
    base_fsu.mark_circle(),
    base_fwt.mark_text(
        align='left',
        baseline='middle',
        dx=15  # Nudges text to right so it doesn't appear on top of the bar
    )
).properties(
    width = width,
    height = height
)

alt.hconcat(
    slope_fsu,
    slope_fwt
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
).resolve_scale(color='independent')

alt.HConcatChart(...)

In [24]:
df = gdp_pc_ppp.loc[
    gdp_pc_ppp.Year.isin([1991, 2020])
].copy()

df.loc[
    df.Region == 'Baltic states',
    'Group'
] = 'FSU-B'

# Add Pct Change and Labels
pct_change = df.groupby('Country Code').apply(
    lambda tx: tx.sort_values(by='Year').loc[:, ['Value']].pct_change().iloc[1] * 100
).rename(columns = {'Value': 'PctChange'})

df = df.merge(pct_change, on = 'Country Code')

df.loc[:, 'Label'] = df.loc[:, 'Country Name'] + ' ↓' + df.loc[:, 'PctChange'].round(2).astype(str) + '%'
df.loc[df.PctChange>0, 'Label'] = df.loc[:, 'Country Name'] + ' ↑+' + df.loc[:, 'PctChange'].round(2).astype(str) + '%'


x = 'Year:O'
y = 'Value'
detail = 'Country Code'

color_fwt = alt.Color(
    'Group',
    scale = alt.Scale(
    domain = ['FWT', 'FSU', 'FSU-B', 'null'],
    range = ['#D50000', '#E0E0E0', '#E0E0E0', '#212121']
    ),
    legend=alt.Legend(orient="top")
)

color_fsu = alt.Color(
    'Group',
    scale = alt.Scale(
    domain = ['FWT', 'FSU', 'FSU-B', 'null'],
    range = ['#E0E0E0', '#D50000', '#E0E0E0', '#212121']
    ),
    legend=alt.Legend(orient="top")
)

color_fsu_b = alt.Color(
    'Group',
    scale = alt.Scale(
    domain = ['FWT', 'FSU', 'FSU-B', 'null'],
    range = ['#E0E0E0', '#E0E0E0', '#D50000', '#212121']
    ),
    legend=alt.Legend(orient="top")
)

text = alt.condition(alt.datum.Year == 2020, 'Label', alt.value(' '))

width = 250
height = 550

# FWT

base_fwt = alt.Chart(
    df.loc[
        ~df['Country Code'].isin(['FSU', 'FSU-B', 'ECS', 'ECA'])
    ]
).encode(
    x=x,
    y=y,
    color=color_fwt,
    detail=detail,
    text=text
)

slope_fwt = alt.layer(
    base_fwt.mark_line(),
    base_fwt.mark_circle(),
    base_fwt.mark_text(
        align='left',
        baseline='middle',
        dx=15  # Nudges text to right so it doesn't appear on top of the bar
    )
).properties(
    width = width,
    height = height
)

# FSU

base_fsu = alt.Chart(
    df.loc[
        ~df['Country Code'].isin(['FWT', 'FSU-B', 'EUU', 'ECA'])
    ]
).encode(
    x=x,
    y=y,
    color=color_fsu,
    detail=detail,
    text=text
)

slope_fsu = alt.layer(
    base_fsu.mark_line(),
    base_fsu.mark_circle(),
    base_fsu.mark_text(
        align='left',
        baseline='middle',
        dx=15  # Nudges text to right so it doesn't appear on top of the bar
    )
).properties(
    width = width,
    height = height
)


# FSU - B

base_fsu_b = alt.Chart(
    df.loc[
        ~df['Country Code'].isin(['FWT', 'FSU', 'ECS', 'ECA'])
    ]
).encode(
    x=x,
    y=y,
    color=color_fsu_b,
    detail=detail,
    text=text
)

slope_fsu_b = alt.layer(
    base_fsu_b.mark_line(),
    base_fsu_b.mark_circle(),
    base_fsu_b.mark_text(
        align='left',
        baseline='middle',
        dx=15  # Nudges text to right so it doesn't appear on top of the bar
    )
).properties(
    width = width,
    height = height
)

alt.hconcat(
    slope_fsu,
    slope_fsu_b,
    slope_fwt
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
).resolve_scale(color='independent', y='shared')

alt.HConcatChart(...)

In [25]:
x = 'Year:O'
y = 'Value'
color = 'Group'
detail = 'Country Code'
width = 250
height = 500

df = gdp_pc_ppp.loc[
    ~gdp_pc_ppp.Group.isin(['FSU', 'ECS', 'ECA']) & \
    gdp_pc_ppp.Year.isin([1990, 2020])
]

base = alt.Chart(df).encode(
    x=x,
    y=y,
#    color=color,
    detail=detail
)

fwt_slope = alt.layer(
    base.mark_line(),
    base.mark_circle()
).properties(
    width = width,
    height = height
)

df = gdp_pc_ppp.loc[
    ~gdp_pc_ppp.Group.isin(['FWT', 'ECS', 'ECA']) & \
    gdp_pc_ppp.Year.isin([1990, 2020])
]

base = alt.Chart(df).encode(
    x=x,
    y=y,
#    color=color,
    detail=detail
)

fsu_slope = alt.layer(
    base.mark_line(),
    base.mark_circle()
).properties(
    width = width,
    height = height
)

alt.hconcat(
    fwt_slope,
    fsu_slope
).configure_axis(
    grid=False
)

alt.HConcatChart(...)

In [26]:
x = 'Country Code'
y = 'Value'
color = 'PeriodChg'
detail = 'Country Code'

chart_df = gdp_pc_ppp.loc[
    gdp_pc_ppp.Year.isin([1990, 2020])
].copy()

change = chart_df.groupby('Country Code').apply(
    lambda tx: tx.sort_values(by='Year').loc[:, ['Value']].diff().iloc[1] > 0
).rename(columns = {'Value': 'PeriodChg'})

chart_df = chart_df.merge(change, on = 'Country Code')

width = 250
height = 500

df = gdp_pc_ppp.loc[
    ~gdp_pc_ppp.Group.isin(['FSU']) & \
    gdp_pc_ppp.Year.isin([1990, 2020])
].copy()

base = alt.Chart(
    chart_df.loc[
        ~chart_df.Group.isin(['FSU'])
    ]
).encode(
    x=x,
    y=y,
    color=color,
    detail=detail
)

fwt_slope = alt.layer(
    base.mark_line(),
    base.mark_circle()
).properties(
    width = width,
    height = height
)

base = alt.Chart(
    chart_df.loc[
        ~chart_df.Group.isin(['FWT'])
    ]
).encode(
    x=x,
    y=y,
    color=color,
    detail=detail
)

fsu_slope = alt.layer(
    base.mark_line(),
    base.mark_circle()
).properties(
    width = width,
    height = height
)

alt.hconcat(
    fwt_slope,
    fsu_slope
)

alt.HConcatChart(...)

### NOT ONLY GDP - UNEMPLOYMENT - WORLD BANK ILO ESTIMATE

In [27]:
# Unemployment, total (% of total labor force) (modeled ILO estimate)
uem_rate = pd.read_csv(
    'data/API_SL.UEM.TOTL.ZS_DS2_en_csv_v2_3629004.csv',
    sep=',',
    skiprows=4,
    usecols = use_cols
)

uem_rate = uem_rate.loc[
    uem_rate['Country Code'].isin(iso_codes + ['EUU', 'ECS', 'ECA'])
]

In [33]:
uem_rate.head()

,Country Name,Country Code,Indicator Code,1990,1991,1992,1993,1994,1995,1996,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
10,Armenia,ARM,SL.UEM.TOTL.ZS,NaN,1.60,1.8,5.30,6.6,6.700000,9.30,...,18.440001,17.299999,16.180,17.500,18.26,17.620001,17.700001,18.969999,18.299999,21.205999
15,Azerbaijan,AZE,SL.UEM.TOTL.ZS,NaN,0.90,1.8,4.50,6.3,7.200000,8.10,...,5.420000,5.190000,4.970,4.910,4.96,5.000000,5.000000,4.900000,4.850000,6.463000
21,Bulgaria,BGR,SL.UEM.TOTL.ZS,NaN,11.10,15.3,16.40,12.8,11.100000,12.50,...,11.260000,12.270000,12.940,11.420,9.14,7.570000,6.160000,5.210000,4.230000,5.120000
25,Belarus,BLR,SL.UEM.TOTL.ZS,NaN,0.60,3.1,8.50,12.8,17.700001,24.40,...,6.166000,6.051000,6.006,5.994,5.84,5.840000,5.650000,4.760000,4.160000,4.768000
54,Czech Republic,CZE,SL.UEM.TOTL.ZS,NaN,2.27,2.6,4.32,4.3,4.020000,3.89,...,6.710000,6.980000,6.950,6.110,5.05,3.950000,2.890000,2.240000,2.010000,2.550000


In [34]:
uem_rate = uem_rate.melt(
    id_vars = ['Country Name', 'Country Code', 'Indicator Code'],
    value_name = 'Value',
    var_name = 'Year'
)

In [37]:
uem_rate.loc[
    uem_rate.Value.isnull()
]

,Country Name,Country Code,Indicator Code,Year,Value
0,Armenia,ARM,SL.UEM.TOTL.ZS,1990,NaN
1,Azerbaijan,AZE,SL.UEM.TOTL.ZS,1990,NaN
2,Bulgaria,BGR,SL.UEM.TOTL.ZS,1990,NaN
3,Belarus,BLR,SL.UEM.TOTL.ZS,1990,NaN
4,Czech Republic,CZE,SL.UEM.TOTL.ZS,1990,NaN
5,Europe & Central Asia (excluding high income),ECA,SL.UEM.TOTL.ZS,1990,NaN
6,Europe & Central Asia,ECS,SL.UEM.TOTL.ZS,1990,NaN
7,Estonia,EST,SL.UEM.TOTL.ZS,1990,NaN
8,European Union,EUU,SL.UEM.TOTL.ZS,1990,NaN
9,Georgia,GEO,SL.UEM.TOTL.ZS,1990,NaN


### Life expectancy at birth, total (years) - World Bank

https://data.worldbank.org/indicator/SP.DYN.LE00.IN

In [38]:
life_exp = pd.read_csv(
    'data/API_SP.DYN.LE00.IN_DS2_en_csv_v2_3630764.csv',
    sep=',',
    skiprows=4,
    usecols = use_cols
)

life_exp = life_exp.loc[
    life_exp['Country Code'].isin(iso_codes + ['EUU', 'ECS', 'ECA'])
]

In [40]:
life_exp = life_exp.melt(
    id_vars = ['Country Name', 'Country Code', 'Indicator Code'],
    value_name = 'Value',
    var_name = 'Year'
)

In [42]:
life_exp.loc[
    life_exp.Value.isnull()
]

,Country Name,Country Code,Indicator Code,Year,Value
720,Armenia,ARM,SP.DYN.LE00.IN,2020,NaN
721,Azerbaijan,AZE,SP.DYN.LE00.IN,2020,NaN
722,Bulgaria,BGR,SP.DYN.LE00.IN,2020,NaN
723,Belarus,BLR,SP.DYN.LE00.IN,2020,NaN
724,Czech Republic,CZE,SP.DYN.LE00.IN,2020,NaN
725,Europe & Central Asia (excluding high income),ECA,SP.DYN.LE00.IN,2020,NaN
726,Europe & Central Asia,ECS,SP.DYN.LE00.IN,2020,NaN
727,Estonia,EST,SP.DYN.LE00.IN,2020,NaN
728,European Union,EUU,SP.DYN.LE00.IN,2020,NaN
729,Georgia,GEO,SP.DYN.LE00.IN,2020,NaN


### GOVERNANCE INDICATORS

### HAPPINESS SCORE